In [7]:
alph = "абвгдежзийклмнопрстуфхцчшщъыьэюя"
p = [0.0801, 0.0159, 0.0454, 0.0165, 0.0298, 0.0845, 0.0899, 0.0254, 0.0716, 0.0007, 0.0186, 0.0523, 0.0312, 0.0670, 0.1097, 0.0281,
         0.0473, 0.0547, 0.0626, 0.0262, 0.0026, 0.0097, 0.0048, 0.0144, 0.0073, 0.0062, 0.0457, 0.0310, 0.0739, 0.0201, 0.0010, 0.0047]

In [8]:
def process_text(text):
    result = ""
    for char in text.lower():
        if char in alph:
            result += char
    return result

In [9]:
def encrypt(text, key):
    result = ""
    k_len = len(key)
    key_i = 0
    for char in text:
        key_char = key[key_i % k_len]
        key_i += 1
        result += alph[(alph.index(char) + alph.index(key_char)) % 32]
    return result

def decrypt(encrypted, key):
    result = ""
    k_len = len(key)
    key_i = 0
    for char in encrypted:
        result += alph[(alph.index(char) - alph.index(key[key_i % k_len])) % 32]
        key_i += 1
    return result


In [10]:
def calculate_ioc(text):
    t_len = len(text)
    freq = [0] * 32
    for char in text:
        freq[alph.index(char)] += 1
    return sum([(f * (f - 1)) / (t_len * (t_len - 1)) for f in freq])



def divide_blocks(text, r):
    blocks = []
    for i in range(r):
        block = text[i::r]
        blocks.append(block)
    return blocks


def key_len(text):
    k_len = {}
    for k in range(2, 32):
        i = 0
        blocks = divide_blocks(text, k)
        for b in blocks:
            i += calculate_ioc(b)
        k_len[k] = i / len(blocks)
   # print(f"{k_len}")
    return max(k_len, key=k_len.get)


In [11]:
def find_key_M_method(text, r):
    key = ""
    blocks = divide_blocks(text, r)
    for i in range(r):
        k = 0
        M = 0
        for j in range(32):
            tempM = 0
            for d in range(32):
                tempM += p[d] * blocks[i].count(alph[(d + j) % 32])
            if tempM > M:
                M = tempM
                k = j
        key += alph[k]
    return key

In [12]:
def main():
    with open("/content/sample_data/to_encrypt.txt", "r") as file:
        text = process_text(file.read())

    keys = [
        'ка',
        'щфл',
        'школ',
        'фгурт',
        'йомайкафрам',
        'ахбольшойрастет',]

    #encrypted_texts = []
    iocs = {}

    for key in keys:
        #encrypted_texts.append(encrypt(text, key))
        #print("ого " + encrypt(text, key))
        iocs[key] = calculate_ioc(encrypt(text, key))

    print(f"IOC: \nioc тексту: {calculate_ioc(text)}")
    for key, ioc in iocs.items():
        print(f"З Ключем \"{key}\" ioc: {ioc}")

    to_decrypt = process_text(open("/content/sample_data/to_decrypt.txt", "r").read())

    key_length = key_len(to_decrypt)
    print("Довжина ключа: " + str(key_length))

    key = find_key_M_method(to_decrypt, key_length)
    print("Ключ: " + key)

    file = open("decrypted.txt", "w")
    file.write(decrypt(to_decrypt, key))
    file.close()
main()

IOC: 
ioc тексту: 0.05626390172124696
З Ключем "ка" ioc: 0.04006091092632516
З Ключем "щфл" ioc: 0.04014988194230572
З Ключем "школ" ioc: 0.0382541149094891
З Ключем "фгурт" ioc: 0.038435478903603326
З Ключем "йомайкафрам" ioc: 0.034151182287923895
З Ключем "ахбольшойрастет" ioc: 0.03323751839304657
Довжина ключа: 14
Ключ: явнодругойключ
